<a href="https://colab.research.google.com/github/x1001000/Colab-Notebooks/blob/main/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 從 [博客來 > 中文書 > 分類總覽頁](https://www.books.com.tw/web/sys_sublistb/books/?loc=subject_012) 觀察到分類範圍有大中小三層，而每個小分類網址中都有XXYYZZ，XX為第一層編號，YY為第二層編號，ZZ為第三層編號，例如 [文學小說 > 翻譯文學 > 日本文學](https://www.books.com.tw/web/sys_bbotm/books/010101) 的網址為 www.books.com.tw/web/sys_bbotm/books/010101
1. 設計三層巢狀迴圈，從010101開始抓資料，到010109發現過頭了，再到010201，依此類推，但這方法不好寫，不如用 [re](https://docs.python.org/3/library/re.html) 直接把分類總覽頁中有連續六個數字的網址全部找出來
1. 有些只有第二層沒有第三層的中分類，例如 [文學小說 > 溫馨/療癒小說](https://www.books.com.tw/web/books_bmidm_0111/?loc=P_0001_2_011) 的網址就沒有XXYYZZ，要設法另外找
1. 用迴圈遍訪找到的分類網址，把每個分類的書籍資料逐頁（100本/頁）彙整在一個 [pandas.DataFrame](https://oranwind.org/python-pandas-ji-chu-jiao-xue/)，最後匯出一個 csv 檔
1. 依序執行以下 code cell 或從 Runtime 選單 Run all，全站爬完要很久，可以隨時按停止，若電腦進入休眠或離線 Runtime 會被收回，但 csv 檔們仍會在 Google Drive
1. 博客來網站會對持續讀取網頁的連線停止回應，若爬蟲繼續嘗試連線則會被封鎖，必須重置 Runtime 換 IP 才能解決，經實測發現，博客來網站停止回應時，讓爬蟲裝死一分鐘，就能重新得到回應

In [ ]:
# 掛載 Google Drive 於目前 Runtime 下，程式匯出的 csv 檔們就可以存進 Google Drive，再用 Google Sheets 開啟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 下 Linux 指令在 Google Drive 新增資料夾
! mkdir drive/MyDrive/博客來-中文書

In [ ]:
# 匯入所需的 Python 套件
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep

In [ ]:
# 找出全部博客來中文書的分類網址
r = requests.get('https://www.books.com.tw/web/sys_sublistb/books/?loc=subject_012')
urls = []
soup = BeautifulSoup(r.text)
for link in soup.find_all('a'):
    url = link.get('href')
    if re.search(r'/\d\d\d\d\d\d/', url):
        urls.append(url)
n = len(urls)
for mid in r.text.split('</th>'):
    if '<ul></ul>' in mid:
        urls.append(BeautifulSoup(mid).find('a').get('href'))
print(f'找到{n}個小分類網址，另外找到{len(urls)-n}個沒有小分類的中分類網址，皆儲存於urls中')


找到945個小分類網址，另外找到64個沒有小分類的中分類網址，皆儲存於urls中


In [ ]:
# 遍訪找到的分類網址，逐頁彙整書籍資料，DataFrame 轉 csv，存至 Google Drive
for url in urls:
    while True:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text)
            filename = soup.title.string.replace('/', '、')
            print(filename, end=' Page:')
            df = pd.DataFrame(columns=['書名', '作者', '出版日期', '內容介紹', '網址'])
            p = 1
            break
        else:
            print('(暫停一分鐘)')
            sleep(60)
    while True:
        r = requests.get(url+f'&o=5&page={p}') # 排序依暢銷度&跳到p頁
        if r.status_code == 200:
            soup = BeautifulSoup(r.text)
            print(p, end='.')
            p += 1
            books = soup.find_all(class_ = 'item')
            for book in books:
                book_info = book.find(class_ = 'info')
                if book_info:
                    df.loc[len(df)+1] = None
                    df.iloc[-1]['書名'] = book.find('h4').text
                    df.iloc[-1]['作者'] = book_info.text.split('，')[0]
                    df.iloc[-1]['出版日期'] = book_info.text.split('，')[-1].split('：')[-1]
                    df.iloc[-1]['內容介紹'] = book.find(class_ = 'txt_cont').text
                    df.iloc[-1]['網址'] = book.find('a').get('href')
            if len(df) < 100 or '共有<em>100</em>本' in r.text: # 分類少於100本或等於100本
                break
        elif r.status_code == 404: # 超過頁數
            break
        else:
            print('(暫停一分鐘)', end='.')
            sleep(60)
    df.to_csv('drive/My Drive/博客來-中文書/'+filename+'.csv')
    print('✅')


博客來-中文書>文學小說>翻譯文學>日本文學 Page:1.2.3.4.5.6.7.8.9.10.11.12.13.14.✅
博客來-中文書>文學小說>翻譯文學>韓國文學 Page:1.✅
博客來-中文書>文學小說>翻譯文學>亞洲文學 Page:1.✅
博客來-中文書>文學小說>翻譯文學>美國文學 Page:1.2.3.4.5.6.7.8.✅
博客來-中文書>文學小說>翻譯文學>南美文學 Page:1.✅
博客來-中文書>文學小說>翻譯文學>英國文學 Page:1.2.3.4.5.✅
(暫停一分鐘)
博客來-中文書>文學小說>翻譯文學>德國文學 Page:1.✅
博客來-中文書>文學小說>翻譯文學>法國文學 Page:1.2.3.✅
博客來-中文書>文學小說>翻譯文學>其他地區 Page:1.2.3.4.✅
博客來-中文書>文學小說>華文創作>散文 Page:1.2.3.4.5.6.7.8.9.10.11.12.13.14.✅
博客來-中文書>文學小說>華文創作>小說 Page:1.2.3.4.5.6.7.8.9.10.(暫停一分鐘).(暫停一分鐘).11.✅
博客來-中文書>文學小說>華文創作>飲食文學 Page:1.✅
博客來-中文書>文學小說>華文創作>旅遊文學 Page:1.✅
博客來-中文書>文學小說>華文創作>自然書寫 Page:1.✅
博客來-中文書>文學小說>華文創作>報導文學 Page:1.✅
博客來-中文書>文學小說>詩>華文現代詩 Page:1.2.3.4.5.✅
博客來-中文書>文學小說>詩>外國詩 Page:1.2.✅
博客來-中文書>文學小說>文學研究>華文文學研究 Page:1.2.3.4.5.✅
博客來-中文書>文學小說>文學研究>外國文學研究 Page:1.2.✅
博客來-中文書>文學小說>文學研究>文學史 Page:1.✅
博客來-中文書>文學小說>文學研究>作家傳記 Page:1.✅
博客來-中文書>文學小說>中國古典文學>經史子集 Page:1.✅
博客來-中文書>文學小說>中國古典文學>古典小說 Page:1.(暫停一分鐘).2.✅
博客來-中文書>文學小說>中國古典文學>古典文學 Page:1.2.✅
博客來-中文書>文學小說>中國古典文學>詩詞曲賦 Page:1.2.3.✅
博客來-中文書>文學小說>國學常識>作文、創作